# Hyperparameter Tuning for Deepfake Detection CNN Model Development

This notebook contains hyperparameter tuning process for CNN model development in deepfake detection. In this case, the tuned hyperparameters includes the number of convolutional layers including their filters and kernel sizes, number of dense layers including their units, dropout rate, and Adam optimizer's initial learning rate.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras_tuner as kt
import pandas as pd
import os

## Define Dataset Directory Path

In [2]:
CDF_TRAIN_DIR = "/kaggle/input/deepfake-detection-datasets/Celeb-DF-v2/Train"
CDF_VAL_DIR = "/kaggle/input/deepfake-detection-datasets/Celeb-DF-v2/Val"

DF_TRAIN_DIR = "/kaggle/input/deepfake-detection-datasets/DeeperForensics-1.0/Train"
DF_VAL_DIR = "/kaggle/input/deepfake-detection-datasets/DeeperForensics-1.0/Val"

DFDC_TRAIN_DIR = "/kaggle/input/deepfake-detection-datasets/DFDC/Train"
DFDC_VAL_DIR = "/kaggle/input/deepfake-detection-datasets/DFDC/Val"

In [3]:
cdf_train_deepfake_dir = os.path.join(CDF_TRAIN_DIR, "Deepfake")
cdf_train_real_vid_dir = os.path.join(CDF_TRAIN_DIR, "Original")
cdf_val_deepfake_dir = os.path.join(CDF_VAL_DIR, "Deepfake")
cdf_val_real_vid_dir = os.path.join(CDF_VAL_DIR, "Original")

df_train_deepfake_dir = os.path.join(DF_TRAIN_DIR, "Deepfake")
df_train_real_vid_dir = os.path.join(DF_TRAIN_DIR, "Original")
df_val_deepfake_dir = os.path.join(DF_VAL_DIR, "Deepfake")
df_val_real_vid_dir = os.path.join(DF_VAL_DIR, "Original")

dfdc_train_deepfake_dir = os.path.join(DFDC_TRAIN_DIR, "Deepfake")
dfdc_train_real_vid_dir = os.path.join(DFDC_TRAIN_DIR, "Original")
dfdc_val_deepfake_dir = os.path.join(DFDC_VAL_DIR, "Deepfake")
dfdc_val_real_vid_dir = os.path.join(DFDC_VAL_DIR, "Original")

In [4]:
print("Dataset loaded:")

print("\nCeleb-DF-v2 (Train Split)")
print(f" - {len(os.listdir(cdf_train_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(cdf_train_real_vid_dir))} real vid frames")
print("Celeb-DF-v2 (Val Split)")
print(f" - {len(os.listdir(cdf_val_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(cdf_val_real_vid_dir))} real vid frames")

print("\nDeeperForensics-1.0 (Train Split)")
print(f" - {len(os.listdir(df_train_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(df_train_real_vid_dir))} real vid frames")
print("DeeperForensics-1.0 (Val Split)")
print(f" - {len(os.listdir(df_val_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(df_val_real_vid_dir))} real vid frames")

print("\nDeepfake Detection Challenge (Train Split)")
print(f" - {len(os.listdir(dfdc_train_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(dfdc_train_real_vid_dir))} real vid frames")
print("Deepfake Detection Challenge (Val Split)")
print(f" - {len(os.listdir(dfdc_val_deepfake_dir))} deepfake frames")
print(f" - {len(os.listdir(dfdc_val_real_vid_dir))} real vid frames")

Dataset loaded:

Celeb-DF-v2 (Train Split)
 - 7000 deepfake frames
 - 7000 real vid frames
Celeb-DF-v2 (Val Split)
 - 1000 deepfake frames
 - 1000 real vid frames

DeeperForensics-1.0 (Train Split)
 - 7000 deepfake frames
 - 7000 real vid frames
DeeperForensics-1.0 (Val Split)
 - 1000 deepfake frames
 - 1000 real vid frames

Deepfake Detection Challenge (Train Split)
 - 7000 deepfake frames
 - 7000 real vid frames
Deepfake Detection Challenge (Val Split)
 - 1000 deepfake frames
 - 1000 real vid frames


## Create Combined Dataset

For hyperparameter tuning purposes, the three datasets are combined into a new set for each train and validation split. For the train split, the full dataset will be sampled, taking only 20% for each dataset to reduce computational cost in the hyperparameter tuning process. The validation split is also sampled, taking only 40% of the data.

### Get Full Filepath for Each Frame

In [5]:
def get_filepath_from_dir(dir_path):
    return [
        os.path.join(dir_path, filename) for filename in sorted(os.listdir(dir_path))
    ]

#### Train Split

In [6]:
cdf_train_deepfake_filepaths = get_filepath_from_dir(cdf_train_deepfake_dir)
cdf_train_real_vid_filepaths = get_filepath_from_dir(cdf_train_real_vid_dir)

df_train_deepfake_filepaths = get_filepath_from_dir(df_train_deepfake_dir)
df_train_real_vid_filepaths = get_filepath_from_dir(df_train_real_vid_dir)

dfdc_train_deepfake_filepaths = get_filepath_from_dir(dfdc_train_deepfake_dir)
dfdc_train_real_vid_filepaths = get_filepath_from_dir(dfdc_train_real_vid_dir)

In [7]:
# Sample the 20% data of the full train split

train_sample_size_per_dataset = 0.2 * len(cdf_train_deepfake_filepaths)
train_sample_size_per_label = int(train_sample_size_per_dataset / 2)

cdf_train_deepfake_filepaths = cdf_train_deepfake_filepaths[:train_sample_size_per_label]
cdf_train_real_vid_filepaths = cdf_train_real_vid_filepaths[:train_sample_size_per_label]

df_train_deepfake_filepaths = df_train_deepfake_filepaths[:train_sample_size_per_label]
df_train_real_vid_filepaths = df_train_real_vid_filepaths[:train_sample_size_per_label]

dfdc_train_deepfake_filepaths = dfdc_train_deepfake_filepaths[:train_sample_size_per_label]
dfdc_train_real_vid_filepaths = dfdc_train_real_vid_filepaths[:train_sample_size_per_label]

#### Val Split

In [8]:
cdf_val_deepfake_filepaths = get_filepath_from_dir(cdf_val_deepfake_dir)
cdf_val_real_vid_filepaths = get_filepath_from_dir(cdf_val_real_vid_dir)

df_val_deepfake_filepaths = get_filepath_from_dir(df_val_deepfake_dir)
df_val_real_vid_filepaths = get_filepath_from_dir(df_val_real_vid_dir)

dfdc_val_deepfake_filepaths = get_filepath_from_dir(dfdc_val_deepfake_dir)
dfdc_val_real_vid_filepaths = get_filepath_from_dir(dfdc_val_real_vid_dir)

In [9]:
# Sample the 40% data of the full train split

val_sample_size_per_dataset = 0.4 * len(cdf_val_deepfake_filepaths)
val_sample_size_per_label = int(val_sample_size_per_dataset / 2)

cdf_val_deepfake_filepaths = cdf_val_deepfake_filepaths[:val_sample_size_per_label]
cdf_val_real_vid_filepaths = cdf_val_real_vid_filepaths[:val_sample_size_per_label]

df_val_deepfake_filepaths = df_val_deepfake_filepaths[:val_sample_size_per_label]
df_val_real_vid_filepaths = df_val_real_vid_filepaths[:val_sample_size_per_label]

dfdc_val_deepfake_filepaths = dfdc_val_deepfake_filepaths[:val_sample_size_per_label]
dfdc_val_real_vid_filepaths = dfdc_val_real_vid_filepaths[:val_sample_size_per_label]

### Combine Dataset in a List

#### Train Split

In [10]:
combined_train_filepaths = []
combined_train_filepaths += cdf_train_deepfake_filepaths + cdf_train_real_vid_filepaths
combined_train_filepaths += df_train_deepfake_filepaths + df_train_real_vid_filepaths
combined_train_filepaths += dfdc_train_deepfake_filepaths + dfdc_train_real_vid_filepaths

# Get labels for each file based on their directory name
combined_train_labels = [
    filepath.split("/")[-2]
    for filepath in combined_train_filepaths
]

print(f"Got {len(combined_train_filepaths)} frames in total")

Got 4200 frames in total


#### Val Split

In [11]:
combined_val_filepaths = []
combined_val_filepaths += cdf_val_deepfake_filepaths + cdf_val_real_vid_filepaths
combined_val_filepaths += df_val_deepfake_filepaths + df_val_real_vid_filepaths
combined_val_filepaths += dfdc_val_deepfake_filepaths + dfdc_val_real_vid_filepaths

# Get labels for each file based on their directory name
combined_val_labels = [
    filepath.split("/")[-2]
    for filepath in combined_val_filepaths
]

print(f"Got {len(combined_val_filepaths)} frames in total")

Got 1200 frames in total


### Create a DataFrame to Store the Dataset

#### Train Split

In [12]:
tuning_train_dataset = pd.DataFrame({
    "filepath": combined_train_filepaths,
    "label": combined_train_labels
})

tuning_train_dataset.head()

,filepath,label
0,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
1,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
2,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
3,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
4,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake


#### Val Split

In [13]:
tuning_val_dataset = pd.DataFrame({
    "filepath": combined_val_filepaths,
    "label": combined_val_labels
})

tuning_val_dataset.head()

,filepath,label
0,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
1,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
2,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
3,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake
4,/kaggle/input/deepfake-detection-datasets/Cele...,Deepfake


## Image Data Generator

In [14]:
def create_generator(dataset_df):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_dataframe(
        dataset_df,
        x_col="filepath",
        y_col="label",
        target_size=(128, 128),
        batch_size=32,
        color_mode="rgb",
        class_mode="binary",
        shuffle=True
    )

    return generator

In [15]:
print("Train image generator created:")
train_generator = create_generator(tuning_train_dataset)

print("\nValidation image generator created:")
val_generator = create_generator(tuning_val_dataset)

Train image generator created:
Found 4200 validated image filenames belonging to 2 classes.

Validation image generator created:
Found 1200 validated image filenames belonging to 2 classes.


## Model Builder Function

In [16]:
def model_builder(hp):
    conv_layers = hp.Int("conv_layers", min_value=3, max_value=7)
    conv_layer_filters = [
        hp.Int(f"conv_{i+1}_filters", min_value=32, max_value=256, step=32)
        for i in range(conv_layers)
    ]
    conv_layer_kernel_size = [
        hp.Int(f"conv_{i+1}_kernel_size", min_value=3, max_value=5, step=2)
        for i in range(conv_layers)
    ]
    dense_layers = hp.Int("dense_layers", min_value=2, max_value=7)
    dense_layer_units = [
        hp.Int(f"dense_{i+1}_layer_units", min_value=32, max_value=128, step=32)
        for i in range(dense_layers)
    ]
    dropout_rate = hp.Float("dropout_rate", min_value=0.4, max_value=0.8, step=0.1)
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-2, step=10)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(128, 128, 3)))

    for i in range(conv_layers):
        model.add(
            tf.keras.layers.Conv2D(
                conv_layer_filters[i],
                (conv_layer_kernel_size[i], conv_layer_kernel_size[i]),
                padding="same",
                activation="relu",
            )
        )
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

    model.add(tf.keras.layers.Flatten())

    for i in range(dense_layers):
        model.add(tf.keras.layers.Dense(dense_layer_units[i], activation="relu"))

    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )

    return model

## Search for the Best Hyperparameters

In [17]:
tuner = kt.Hyperband(model_builder,
                     objective="val_accuracy",
                     max_epochs=20,
                     factor=3,
                     directory="tuner-result",
                     project_name="cnn-best-hp")

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=5,
                                                  restore_best_weights=True)

In [19]:
tuner.search(train_generator,
             validation_data=val_generator,
             verbose=1,
             callbacks=[early_stopping])

Trial 30 Complete [00h 02m 42s]
val_accuracy: 0.6000000238418579

Best val_accuracy So Far: 0.7691666483879089
Total elapsed time: 00h 36m 53s


In [20]:
tuner.results_summary()

Results summary
Results in tuner-result/cnn-best-hp
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0024 summary
Hyperparameters:
conv_layers: 6
conv_1_filters: 256
conv_2_filters: 32
conv_3_filters: 160
conv_1_kernel_size: 3
conv_2_kernel_size: 5
conv_3_kernel_size: 5
dense_layers: 6
dense_1_layer_units: 128
dense_2_layer_units: 96
dropout_rate: 0.6000000000000001
learning_rate: 1e-05
dense_3_layer_units: 96
dense_4_layer_units: 96
dense_5_layer_units: 64
conv_4_filters: 128
conv_4_kernel_size: 5
conv_5_filters: 192
conv_6_filters: 128
conv_7_filters: 224
conv_5_kernel_size: 5
conv_6_kernel_size: 5
conv_7_kernel_size: 3
dense_6_layer_units: 128
dense_7_layer_units: 32
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.7691666483879089

Trial 0026 summary
Hyperparameters:
conv_layers: 7
conv_1_filters: 96
conv_2_filters: 192
conv_3_filters: 96
conv_1_kernel_size: 5
conv_2_kernel_size: 5
conv_3_kernel_size: